In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
acc_cas = {}
for i in range(2012, 2017):
    idx = str(i)
    acc = pd.read_excel('./input/{0}/Acc_cas {0}.xls'.format(idx))
    acc = acc[['ACC_TIME', 'WEEK_DAY', 'RAIN', 'SPEED_LMT', 'TRAFF_CONG', 'JCN_CTRL',
               'RD_TYPE', 'CAS_AGE', 'CAS_SEX', 'INJURY', 'ROLE', 'SB_WORN', 'SEAT', 
               'PED_LOCATN']]

    acc['YEAR'] = idx
    acc_cas[idx] = acc
    print('The Year %s is read.' % idx)

The Year 2012 is read.
The Year 2013 is read.
The Year 2014 is read.
The Year 2015 is read.
The Year 2016 is read.


In [3]:
acc = acc_cas['2012'].copy()
for i in range(2013, 2017):
    acc = acc.append(acc_cas[str(i)].copy())

In [4]:
def str_append(x):
    if isinstance(x, str):
        counter = ['0' for i in range(4 - len(x))]
        counter = ''.join(counter)
        x = counter + x
        return x
    else:
        raise TypeError('The function only support string-type input.')


acc['ACC_TIME'] = acc['ACC_TIME'].map(
    lambda x: str_append(str(x)))
acc['HOUR'] = pd.to_datetime(
    acc['ACC_TIME'], format='%H%M').dt.hour
t0 = acc['HOUR'][acc['HOUR'].isin(
    [7, 8, 9])].count()  # 700am-959am
t1 = acc['HOUR'][acc['HOUR'].isin(
    [10, 11, 12, 13, 14, 15])].count()  # 1000am-0359pm
t2 = acc['HOUR'][acc['HOUR'].isin(
    [16, 17, 18])].count()  # 0400pm-0659pm
t3 = acc['HOUR'][acc['HOUR'].isin(
    [19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6])].count()  # 0700pm-0659am
print('Time: Count(Proportion)')
print('7:00 - 9:59 a.m.: %d(%.1f%%)' %
      (t0, t0/acc['HOUR'].count()*100))
print('10:00 a.m. - 3:59 p.m.: %d(%.1f%%)' %
      (t1, t1/acc['HOUR'].count()*100))
print('4:00 - 6:59 p.m.: %d(%.1f%%)' %
      (t2, t2/acc['HOUR'].count()*100))
print('7:00 p.m. - 6:59 a.m.: %d(%.1f%%)' %
      (t3, t3/acc['HOUR'].count()*100))

Time: Count(Proportion)
7:00 - 9:59 a.m.: 15596(15.4%)
10:00 a.m. - 3:59 p.m.: 33959(33.6%)
4:00 - 6:59 p.m.: 20287(20.1%)
7:00 p.m. - 6:59 a.m.: 31331(31.0%)


In [5]:
weekday_row = acc['WEEK_DAY'].isin([1, 2, 3, 4, 5])
weekday = acc['WEEK_DAY'][weekday_row].count()
weekend = acc['WEEK_DAY'][~weekday_row].count()
print('Day of week: Count(Proportion)')
print('Monday - Friday: %d(%.1f%%)' %
      (weekday, weekday/acc['WEEK_DAY'].count()*100))
print('Weekend: %d(%.1f%%)' %
      (weekend, weekend/acc['WEEK_DAY'].count()*100))

Day of week: Count(Proportion)
Monday - Friday: 70624(69.8%)
Weekend: 30549(30.2%)


In [6]:
r0 = acc['RAIN'][acc['RAIN'] == 1].count()
r1 = acc['RAIN'][acc['RAIN'] == 2].count()
r2 = acc['RAIN'][acc['RAIN'] == 3].count()
r3 = acc['RAIN'][acc['RAIN'].isin([9, 0])].count()

print('Rain: Count(Proportion)')
print('No Rain: %d(%.1f%%)' % (r0, r0/acc['RAIN'].count()*100))
print('Light Rain: %d(%.1f%%)' % (r1, r1/acc['RAIN'].count()*100))
print('Heavy Rain: %d(%.1f%%)' % (r2, r2/acc['RAIN'].count()*100))
print('Unknown: %d(%.1f%%)' % (r3, r3/acc['RAIN'].count()*100))

Rain: Count(Proportion)
No Rain: 86443(85.4%)
Light Rain: 12315(12.2%)
Heavy Rain: 2158(2.1%)
Unknown: 257(0.3%)


In [7]:
s0 = acc['SPEED_LMT'][acc['SPEED_LMT'] == 50].count()
s1 = acc['SPEED_LMT'][acc['SPEED_LMT'] < 50].count()
s2 = acc['SPEED_LMT'][acc['SPEED_LMT'] > 50].count()

print('Speed Limit: Count(Proportion)')
print('50km/h: %d(%.1f%%)' % (s0, s0/acc['SPEED_LMT'].count()*100))
print('Under 50km/h: %d(%.1f%%)' % (s1, s1/acc['SPEED_LMT'].count()*100))
print('Above 50km/h: %d(%.1f%%)' % (s2, s2/acc['SPEED_LMT'].count()*100))

Speed Limit: Count(Proportion)
50km/h: 86065(85.1%)
Under 50km/h: 2039(2.0%)
Above 50km/h: 13069(12.9%)


In [8]:
t0 = acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 3].count()
t1 = acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 1].count()
t2 = acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 2].count()
t3 = acc['TRAFF_CONG'][acc['TRAFF_CONG'].isin([9, 0])].count()

print('Traffic Congestion: Count(Proportion)')
print('None: %d(%.1f%%)' % (t0, t0/acc['TRAFF_CONG'].count()*100))
print('Severe Congestion: %d(%.1f%%)' % (t1, t1/acc['TRAFF_CONG'].count()*100))
print('Moderate Congestion: %d(%.1f%%)' % (t2, t2/acc['TRAFF_CONG'].count()*100))
print('Unknown: %d(%.1f%%)' % (t3, t3/acc['TRAFF_CONG'].count()*100))

Traffic Congestion: Count(Proportion)
None: 55863(55.2%)
Severe Congestion: 15458(15.3%)
Moderate Congestion: 28381(28.1%)
Unknown: 1471(1.5%)


In [9]:
j0 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 6].count()
j1 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 4].count()
j2 = acc['JCN_CTRL'][acc['JCN_CTRL'].isin([2, 3, 5])].count()
j3 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 1].count()
j4 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 0].count()

print('Traffic Congestion: Count(Proportion)')
print('Non Junction: %d(%.1f%%)' % (j0, j0/acc['JCN_CTRL'].count()*100))
print('Traffic Signal: %d(%.1f%%)' % (j1, j1/acc['JCN_CTRL'].count()*100))
print('Other Control Types: %d(%.1f%%)' % (j2, j2/acc['JCN_CTRL'].count()*100))
print('No Control: %d(%.1f%%)' % (j3, j3/acc['JCN_CTRL'].count()*100))
print('Unknown: %d(%.1f%%)' % (j4, j4/acc['JCN_CTRL'].count()*100))

Traffic Congestion: Count(Proportion)
Non Junction: 77440(76.5%)
Traffic Signal: 13228(13.1%)
Other Control Types: 4762(4.7%)
No Control: 5711(5.6%)
Unknown: 32(0.0%)


In [10]:
r0 = acc['RD_TYPE'][acc['RD_TYPE'].isin([3, 4])].count()
r1 = acc['RD_TYPE'][acc['RD_TYPE'] == 1].count()
r2 = acc['RD_TYPE'][acc['RD_TYPE'] == 2].count()
r3 = acc['RD_TYPE'][acc['RD_TYPE'] == 0].count()

print('Traffic Congestion: Count(Proportion)')
print('Multi-/dual Carriageway: %d(%.1f%%)' % (r0, r0/acc['RD_TYPE'].count()*100))
print('One-way Carriageway: %d(%.1f%%)' % (r1, r1/acc['RD_TYPE'].count()*100))
print('Two-way Carriageway: %d(%.1f%%)' % (r2, r2/acc['RD_TYPE'].count()*100))
print('Unknown: %d(%.1f%%)' % (r3, r3/acc['RD_TYPE'].count()*100))

Traffic Congestion: Count(Proportion)
Multi-/dual Carriageway: 24412(24.1%)
One-way Carriageway: 41196(40.7%)
Two-way Carriageway: 35558(35.1%)
Unknown: 7(0.0%)


In [11]:
# split out the values of unknown
unknown_row = acc['CAS_AGE'].isin([0, 99, 999])
unknown = acc['CAS_AGE'][unknown_row]
dta_age = acc['CAS_AGE'][~unknown_row]

age_grouped = pd.cut(dta_age, [0, 15, 65, 120], labels=['<15', '15-65', '>65'])
age0 = age_grouped.value_counts()['<15']
age1 = age_grouped.value_counts()['15-65']
age2 = age_grouped.value_counts()['>65']
ageunknown = unknown.count()
print('Age: Count(Proportion)')
print('Under 15: %d(%.1f%%)' %
      (age0, age0/acc['CAS_AGE'].count()*100))
print('15 - 65: %d(%.1f%%)' %
      (age1, age1/acc['CAS_AGE'].count()*100))
print('Above 65: %d(%.1f%%)' %
      (age2, age2/acc['CAS_AGE'].count()*100))
print('Unknown: %d(%.1f%%)' %
      (ageunknown, ageunknown/acc['CAS_AGE'].count()*100))

Age: Count(Proportion)
Under 15: 6674(6.6%)
15 - 65: 84305(83.3%)
Above 65: 9125(9.0%)
Unknown: 1066(1.1%)


In [12]:
sex0 = acc['CAS_SEX'].value_counts()[1]
sex1 = acc['CAS_SEX'].value_counts()[2]
sex2 = acc['CAS_SEX'].value_counts()[9]
print('Sex: Count(Proportion)')
print('Male: %d(%.1f%%)' %
      (sex0, sex0/acc['CAS_SEX'].count()*100))
print('Famale: %d(%.1f%%)' %
      (sex1, sex1/acc['CAS_SEX'].count()*100))
print('Unknown: %d(%.1f%%)' %
      (sex2, sex2/acc['CAS_SEX'].count()*100))

Sex: Count(Proportion)
Male: 63618(62.9%)
Famale: 37439(37.0%)
Unknown: 116(0.1%)


In [13]:
injury0 = acc['INJURY'].value_counts()[1] + acc['INJURY'].value_counts()[2]
injury1 = acc['INJURY'].value_counts()[3]
print('Injury Severity: Count(Proportion)')
print('Killed or severe injury: %d(%.1f%%)' %
      (injury0, injury0/acc['INJURY'].count()*100))
print('Slight Injury: %d(%.1f%%)' %
      (injury1, injury1/acc['INJURY'].count()*100))

Injury Severity: Count(Proportion)
Killed or severe injury: 13565(13.4%)
Slight Injury: 87608(86.6%)


In [14]:
sb0 = acc['SB_WORN'][acc['SB_WORN'] == 1].count()
sb1 = acc['SB_WORN'][acc['SB_WORN'] == 2].count()
sb2 = acc['SB_WORN'][acc['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/acc['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/acc['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/acc['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 63143(62.4%)
No: 18650(18.4%)
Unknown: 19380(19.2%)


In [15]:
# Split into three group to check these variables proportion
accd = acc[acc['ROLE'] == 1]
accp = acc[acc['ROLE'] == 2]
accpd = acc[acc['ROLE'] == 3]

In [16]:
sb0 = accd['SB_WORN'][accd['SB_WORN'] == 1].count()
sb1 = accd['SB_WORN'][accd['SB_WORN'] == 2].count()
sb2 = accd['SB_WORN'][accd['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/accd['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/accd['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/accd['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 39749(80.9%)
No: 8868(18.0%)
Unknown: 544(1.1%)


In [17]:
sb0 = accp['SB_WORN'][accp['SB_WORN'] == 1].count()
sb1 = accp['SB_WORN'][accp['SB_WORN'] == 2].count()
sb2 = accp['SB_WORN'][accp['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/accp['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/accp['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/accp['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 23394(68.9%)
No: 9782(28.8%)
Unknown: 774(2.3%)


In [18]:
sb0 = accpd['SB_WORN'][accpd['SB_WORN'] == 1].count()
sb1 = accpd['SB_WORN'][accpd['SB_WORN'] == 2].count()
sb2 = accpd['SB_WORN'][accpd['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/accpd['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/accpd['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/accpd['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 0(0.0%)
No: 0(0.0%)
Unknown: 18062(100.0%)


In [19]:
s0 = acc['SEAT'][acc['SEAT'] == 3].count()
s1 = acc['SEAT'][acc['SEAT'] == 2].count()
s2 = acc['SEAT'][acc['SEAT'] == 1].count()
s3 = acc['SEAT'][acc['SEAT'].isin([4, 8])].count()
s4 = accd['SEAT'][accd['SEAT'].isin([6, 7])].count()
s5 = accd['SEAT'][accd['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/acc['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/acc['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/acc['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/acc['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/acc['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/acc['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 49161(48.6%)
Front Nearside: 6202(6.1%)
Rear: 20230(20.0%)
Standing: 6408(6.3%)
Others: 0(0.0%)
Unknown: 0(0.0%)


In [20]:
s0 = accd['SEAT'][accd['SEAT'] == 3].count()
s1 = accd['SEAT'][accd['SEAT'] == 2].count()
s2 = accd['SEAT'][accd['SEAT'] == 1].count()
s3 = accd['SEAT'][accd['SEAT'].isin([4, 8])].count()
s4 = accd['SEAT'][accd['SEAT'].isin([6, 7])].count()
s5 = accd['SEAT'][accd['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/accd['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/accd['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/accd['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/accd['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/accd['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/accd['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 49161(100.0%)
Front Nearside: 0(0.0%)
Rear: 0(0.0%)
Standing: 0(0.0%)
Others: 0(0.0%)
Unknown: 0(0.0%)


In [21]:
s0 = accp['SEAT'][accp['SEAT'] == 3].count()
s1 = accp['SEAT'][accp['SEAT'] == 2].count()
s2 = accp['SEAT'][accp['SEAT'] == 1].count()
s3 = accp['SEAT'][accp['SEAT'].isin([4, 8])].count()
s4 = accp['SEAT'][accp['SEAT'].isin([6, 7])].count()
s5 = accp['SEAT'][accp['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/accp['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/accp['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/accp['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/accp['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/accp['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/accp['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 0(0.0%)
Front Nearside: 6202(18.3%)
Rear: 20230(59.6%)
Standing: 6408(18.9%)
Others: 20(0.1%)
Unknown: 1090(3.2%)


In [22]:
s0 = accpd['SEAT'][accpd['SEAT'] == 3].count()
s1 = accpd['SEAT'][accpd['SEAT'] == 2].count()
s2 = accpd['SEAT'][accpd['SEAT'] == 1].count()
s3 = accpd['SEAT'][accpd['SEAT'].isin([4, 8])].count()
s4 = accpd['SEAT'][accpd['SEAT'].isin([6, 7])].count()
s5 = accpd['SEAT'][accpd['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/accpd['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/accpd['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/accpd['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/accpd['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/accpd['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/accpd['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 0(0.0%)
Front Nearside: 0(0.0%)
Rear: 0(0.0%)
Standing: 0(0.0%)
Others: 0(0.0%)
Unknown: 18062(100.0%)


In [23]:
p0 = acc['PED_LOCATN'][acc['PED_LOCATN'] == 3].count()
p1 = acc['PED_LOCATN'][acc['PED_LOCATN'] == 4].count()
p2 = acc['PED_LOCATN'][acc['PED_LOCATN'].isin([1, 2, 5, 8])].count()
p3 = acc['PED_LOCATN'][acc['PED_LOCATN'].isin([0, 9])].count()

print('Pedestrian Location: Count(Proportion)')
print('On controlled crossing: %d(%.1f%%)' % (p0, p0/acc['PED_LOCATN'].count()*100))
print('Within 15M of controlled crossing: %d(%.1f%%)' % (p1, p1/acc['PED_LOCATN'].count()*100))
print('Others: %d(%.1f%%)' % (p2, p2/acc['PED_LOCATN'].count()*100))
print('Unknown: %d(%1.f%%)' % (p3, p3/acc['PED_LOCATN'].count()*100))

Pedestrian Location: Count(Proportion)
On controlled crossing: 2591(2.6%)
Within 15M of controlled crossing: 1059(1.0%)
Others: 14134(14.0%)
Unknown: 83389(82%)


In [24]:
pp0 = accpd['PED_LOCATN'][accpd['PED_LOCATN'] == 3].count()
pp1 = accpd['PED_LOCATN'][accpd['PED_LOCATN'] == 4].count()
pp2 = accpd['PED_LOCATN'][accpd['PED_LOCATN'].isin([1, 2, 5, 8])].count()
pp3 = accpd['PED_LOCATN'][accpd['PED_LOCATN'].isin([0, 9])].count()

print('Pedestrian Location: Count(Proportion)')
print('On controlled crossing: %d(%.1f%%)' % (pp0, pp0/accpd['PED_LOCATN'].count()*100))
print('Within 15M of controlled crossing: %d(%.1f%%)' % (pp1, pp1/accpd['PED_LOCATN'].count()*100))
print('Others: %d(%.1f%%)' % (pp2, pp2/accpd['PED_LOCATN'].count()*100))
print('Unkown: %d(%1.f%%)' % (pp3, pp3/accpd['PED_LOCATN'].count()*100))

Pedestrian Location: Count(Proportion)
On controlled crossing: 2591(14.3%)
Within 15M of controlled crossing: 1059(5.9%)
Others: 14134(78.3%)
Unkown: 278(2%)


In [25]:
print(acc['YEAR'].value_counts())
print(acc['YEAR'].value_counts(normalize=True))

2013    20596
2015    20381
2012    20210
2016    20132
2014    19854
Name: YEAR, dtype: int64
2013    0.203572
2015    0.201447
2012    0.199757
2016    0.198986
2014    0.196238
Name: YEAR, dtype: float64


In [26]:
# ACC_TIME
acc['ACC_TIME'] = acc['ACC_TIME'].map(
    lambda x: str_append(str(x)))
acc['HOUR'] = pd.to_datetime(acc['ACC_TIME'], format='%H%M').dt.hour
# Split into four group,
acc['HOUR'][acc['HOUR'].isin([7, 8, 9])] = '7:00 - 9:59 a.m.'
acc['HOUR'][acc['HOUR'].isin(
    [10, 11, 12, 13, 14, 15])] = '10:00 a.m. - 3:59 p.m.'
acc['HOUR'][acc['HOUR'].isin([16, 17, 18])] = '4:00 - 6:59 p.m.'
acc['HOUR'][acc['HOUR'].isin(
    [19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6])] = '7:00 p.m. - 6:59 a.m.'
# Convert to dummy variables
acc['HOUR'][acc['HOUR'] == '7:00 p.m. - 6:59 a.m.'] = 0
acc['HOUR'][acc['HOUR'] == '7:00 - 9:59 a.m.'] = 1
acc['HOUR'][acc['HOUR'] == '10:00 a.m. - 3:59 p.m.'] = 2
acc['HOUR'][acc['HOUR'] == '4:00 - 6:59 p.m.'] = 3
# Merge into acc
dummy = pd.get_dummies(acc['HOUR'], drop_first=True).rename(columns={1: '7:00 - 9:59 a.m.',
                                                                     2: '10:00 a.m. - 3:59 p.m.',
                                                                     3: '4:00 - 6:59 p.m.'})
acc = pd.concat([acc, dummy], axis=1).drop(['HOUR', 'ACC_TIME'], axis=1)

In [27]:
# Similarly, group and dummy WEEK_DAY,
# Split into two group, weekday and weekend, and Convert to dummy variables
acc['WEEK_DAY'][acc['WEEK_DAY'].isin([1, 2, 3, 4, 5])] = 1
acc['WEEK_DAY'][acc['WEEK_DAY'].isin([6, 7])] = 0
# Merge into acc
dummy = pd.get_dummies(acc['WEEK_DAY'], drop_first=True).rename(columns={1: 'Monday - Friday'})
acc = pd.concat([acc, dummy], axis=1).drop('WEEK_DAY', axis=1)

In [28]:
# Similarly, group and dummy RAIN
# Remove Unknown rows,
acc = acc[~acc['RAIN'].isin([9, 0])]
# Convert to dummy variables
acc['RAIN'][acc['RAIN'] == 1] = 0
acc['RAIN'][acc['RAIN'] == 2] = 1
acc['RAIN'][acc['RAIN'] == 3] = 2
# Merge into acc
dummy = pd.get_dummies(acc['RAIN'], drop_first=True).rename(columns={1: 'Light rain',
                                                                     2: 'Heavy rain'})
acc = pd.concat([acc, dummy], axis=1).drop('RAIN', axis=1)

In [29]:
# Similarly, group and dummy SPEED_LMT
# Convert to dummy variables
acc['SPEED_LMT'][acc['SPEED_LMT'] < 50] = 1
acc['SPEED_LMT'][acc['SPEED_LMT'] > 50] = 0
acc['SPEED_LMT'][acc['SPEED_LMT'] == 50] = 2
# Merge into acc
dummy = pd.get_dummies(acc['SPEED_LMT'], drop_first=True).rename(columns={1: 'Below 50 km/h',
                                                                          2: '50 km/h'})
acc = pd.concat([acc, dummy], axis=1).drop('SPEED_LMT', axis=1)

In [30]:
# Similarly, group and dummy TRAFF_CONG
# Remove Unknown rows,
acc = acc[~acc['TRAFF_CONG'].isin([9, 0])]
# Convert to dummy variables
acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 3] = 0
# Merge into acc
dummy = pd.get_dummies(acc['TRAFF_CONG'], drop_first=True).rename(columns={1: 'Severe Congestion',
                                                                           2: 'Moderate Congestion'})
acc = pd.concat([acc, dummy], axis=1).drop('TRAFF_CONG', axis=1)

In [31]:
# Similarly, group and dummy JCN_CTRL
# Remove Unknown rows,
acc = acc[~acc['JCN_CTRL'].isin([9, 0])]
# Convert to dummy variables
acc['JCN_CTRL'][acc['JCN_CTRL'].isin([2, 3, 5])] = 2
acc['JCN_CTRL'][acc['JCN_CTRL'] == 1] = 3
acc['JCN_CTRL'][acc['JCN_CTRL'] == 4] = 1
acc['JCN_CTRL'][acc['JCN_CTRL'] == 6] = 0

# Merge into acc
dummy = pd.get_dummies(acc['JCN_CTRL'], drop_first=True).rename(columns={1: 'Traffic Signal',
                                                                         2: 'Other Control Types',
                                                                         3: 'No Control'})
acc = pd.concat([acc, dummy], axis=1).drop('JCN_CTRL', axis=1)

In [32]:
# Similarly, group and dummy RD_TYPE
# Remove Unknown rows
acc = acc[~acc['RD_TYPE'].isin([9, 0])]
# Convert to dummy variables
acc['RD_TYPE'][acc['RD_TYPE'].isin([3, 4])] = 0
# Merge into acc
dummy = pd.get_dummies(acc['RD_TYPE'], drop_first=True).rename(columns={1: 'One-way Carriageway',
                                                                        2: 'Two-way Carriageway'})
acc = pd.concat([acc, dummy], axis=1).drop('RD_TYPE', axis=1)

In [33]:
# Similarly, group and dummy CAS_AGE
# Remove Unknown rows
acc = acc[~acc['CAS_AGE'].isin([0, 99, 999])]
# Convert to dummy variables
acc['CAS_AGE'] = pd.cut(acc['CAS_AGE'], [0, 15, 66, 120], labels=[0, 1, 2])
# Merge into acc
dummy = pd.get_dummies(acc['CAS_AGE']).rename(columns={0: 'Under 15',
                                                       1: '15 - 65',
                                                       2: 'Above 65'})
acc = pd.concat([acc, dummy], axis=1).drop(['CAS_AGE', '15 - 65'], axis=1)

In [34]:
# Similarly, group and dummy CAS_SEX
# Remove Unknown rows
acc = acc[~acc['CAS_SEX'].isin([0, 9])]
# Convert to dummy variables
acc['CAS_SEX'][acc['CAS_SEX'] == 2] = 0
# Merge into acc
dummy = pd.get_dummies(acc['CAS_SEX'], drop_first=True).rename(columns={1: 'Male'})
acc = pd.concat([acc, dummy], axis=1).drop('CAS_SEX', axis=1)

In [35]:
# Similarly, group and dummy Year
dummy = pd.get_dummies(acc['YEAR'], prefix='YEAR').drop('YEAR_2016', axis=1)
acc = pd.concat([acc, dummy], axis=1).drop(['YEAR'], axis=1)

In [36]:
# Group INJURY for binary classification
acc['INJURY'][acc['INJURY'].isin([1, 2])] = 1
acc['INJURY'][acc['INJURY'] == 3] = 0
# which means the model would find the prob of seriously injury with the given conditions.

In [37]:
acc.head()

,INJURY,ROLE,SB_WORN,SEAT,PED_LOCATN,7:00 - 9:59 a.m.,10:00 a.m. - 3:59 p.m.,4:00 - 6:59 p.m.,Monday - Friday,Light rain,...,No Control,One-way Carriageway,Two-way Carriageway,Under 15,Above 65,Male,YEAR_2012,YEAR_2013,YEAR_2014,YEAR_2015
0,0,3,0,0,2,0,0,0,0,0,...,0,0,1,0,0,1,1,0,0,0
1,0,2,1,2,0,0,0,0,0,0,...,1,1,0,0,0,1,1,0,0,0
2,0,1,1,3,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
3,0,1,1,3,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
4,0,1,1,3,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [38]:
acc.columns

Index(['INJURY', 'ROLE', 'SB_WORN', 'SEAT', 'PED_LOCATN', '7:00 - 9:59 a.m.',
       '10:00 a.m. - 3:59 p.m.', '4:00 - 6:59 p.m.', 'Monday - Friday',
       'Light rain', 'Heavy rain', 'Below 50 km/h', '50 km/h',
       'Severe Congestion', 'Moderate Congestion', 'Traffic Signal',
       'Other Control Types', 'No Control', 'One-way Carriageway',
       'Two-way Carriageway', 'Under 15', 'Above 65', 'Male', 'YEAR_2012',
       'YEAR_2013', 'YEAR_2014', 'YEAR_2015'],
      dtype='object')

In [39]:
# split dataset to into 3 group, driver, passenger and pedestrian

In [40]:
acc_driver = acc[acc['ROLE'] == 1].drop(['ROLE', 'SEAT', 'PED_LOCATN'], axis=1)
acc_passen = acc[acc['ROLE'] == 2].drop(['ROLE', 'PED_LOCATN'], axis=1)
acc_pedest = acc[acc['ROLE'] == 3].drop(['ROLE', 'SEAT', 'SB_WORN'], axis=1)

In [41]:
# Similarly, group and dummy SB_WORN, but this is only for driver and passen
# Remove Unknown rows
acc_driver = acc_driver[~acc_driver['SB_WORN'].isin([0, 9])]
acc_passen = acc_passen[~acc_passen['SB_WORN'].isin([0, 9])]
# Convert to dummy variables
acc_driver['SB_WORN'][acc_driver['SB_WORN'] == 2] = 0
acc_passen['SB_WORN'][acc_passen['SB_WORN'] == 2] = 0
# Merge into acc
dummy = pd.get_dummies(acc_driver['SB_WORN'], drop_first=True).rename(columns={1: 'With belt or helmet'})
acc_driver = pd.concat([acc_driver, dummy], axis=1).drop('SB_WORN', axis=1)
dummy = pd.get_dummies(acc_passen['SB_WORN'], drop_first=True).rename(columns={1: 'With belt or helmet'})
acc_passen = pd.concat([acc_passen, dummy], axis=1).drop('SB_WORN', axis=1)

In [42]:
# Similarly, group and dummy SEAT, but this is only for passen
# Convert to dummy variables
acc_passen['SEAT'][acc_passen['SEAT'].isin([0, 6, 7, 9])] = 5
acc_passen['SEAT'][acc_passen['SEAT'].isin([4, 8])] = 4
acc_passen['SEAT'][acc_passen['SEAT'] == 3] = 0
# Merge into acc
# rmk Control variables is Driver/Rider
dummy = pd.get_dummies(acc_passen['SEAT'], drop_first=True).rename(
    columns={1: 'Rear', 2: 'Front nearside', 4: 'Standing', 5: 'Others'})
acc_passen = pd.concat([acc_passen, dummy], axis=1).drop('SEAT', axis=1)

In [43]:
# Similarly, group and dummy PED_LOCATN, but this is only for pedest
acc_pedest['PED_LOCATN'][acc_pedest['PED_LOCATN'].isin([0, 1, 2, 5, 8, 9])] = 0
acc_pedest['PED_LOCATN'][acc_pedest['PED_LOCATN'] == 3] = 1
acc_pedest['PED_LOCATN'][acc_pedest['PED_LOCATN'] == 4] = 2
# Merge into acc
# rmk Control variables is other
dummy = pd.get_dummies(acc_pedest['PED_LOCATN'], drop_first=True).rename(
    columns={1: 'On controlled crossing', 2: 'Within 15M of controlled crossing'})
acc_pedest = pd.concat([acc_pedest, dummy], axis=1).drop('PED_LOCATN', axis=1)

In [44]:
X_driver, y_driver = acc_driver.drop('INJURY', axis=1), acc_driver['INJURY']
X_passen, y_passen = acc_passen.drop('INJURY', axis=1), acc_passen['INJURY']
X_pedest, y_pedest = acc_pedest.drop('INJURY', axis=1), acc_pedest['INJURY']

In [45]:
X_driver = sm.add_constant(X_driver)
m_driver = sm.Logit(y_driver, X_driver)
r_driver = m_driver.fit()

Optimization terminated successfully.
         Current function value: 0.389790
         Iterations 6


In [46]:
r_driver.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 INJURY   No. Observations:                47838
Model:                          Logit   Df Residuals:                    47814
Method:                           MLE   Df Model:                           23
Date:                Sat, 14 Apr 2018   Pseudo R-squ.:                 0.02512
Time:                        12:17:25   Log-Likelihood:                -18647.
converged:                       True   LL-Null:                       -19127.
                                        LLR p-value:                2.714e-188
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -1.2418      0.073    -17.080      0.000      -1.384      -1.099
7:00 - 9:59 a.m.          -0.2119      0.044     -4.860      0.000      -0.297      -0.126
10:00 a.m. - 3:59 p.m.    -0.1349      0.033     -4.067      0.000      -0.200      -0.070
4:00 - 6:59 p.m.          -0.2099      0.039     -5.434      0.000      -0.286      -0.134
Monday - Friday           -0.1012      0.029     -3.545      0.000      -0.157      -0.045
Light rain                -0.1466      0.043     -3.428      0.001      -0.230      -0.063
Heavy rain                -0.1491      0.100     -1.496      0.135      -0.344       0.046
Below 50 km/h             -0.3711      0.100     -3.724      0.000      -0.566      -0.176
50 km/h                   -0.2463      0.041     -6.009      0.000      -0.327      -0.166
Severe Congestion         -0.2065      0.042     -4.907      0.000      -0.289      -0.124
Moderate Congestion       -0.2180      0.034     -6.430      0.000      -0.284      -0.152
Traffic Signal            -0.3265      0.052     -6.259      0.000      -0.429      -0.224
Other Control Types       -0.2913      0.072     -4.066      0.000      -0.432      -0.151
No Control                -0.3350      0.077     -4.378      0.000      -0.485      -0.185
One-way Carriageway       -0.0707      0.039     -1.823      0.068      -0.147       0.005
Two-way Carriageway        0.3266      0.038      8.542      0.000       0.252       0.402
Under 15                  -0.4934      0.082     -6.020      0.000      -0.654      -0.333
Above 65                   0.6262      0.073      8.544      0.000       0.483       0.770
Male                       0.1894      0.041      4.664      0.000       0.110       0.269
YEAR_2012                  0.0426      0.043      0.983      0.326      -0.042       0.128
YEAR_2013                  0.0743      0.043      1.729      0.084      -0.010       0.158
YEAR_2014                  0.1413      0.043      3.321      0.001       0.058       0.225
YEAR_2015                  0.0851      0.042      2.007      0.045       0.002       0.168
With belt or helmet       -0.5115      0.034    -14.897      0.000      -0.579      -0.444
==========================================================================================
"""

In [47]:
p_driver = r_driver.params
c_driver = r_driver.conf_int()
c_driver['OR'] = p_driver
c_driver.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(c_driver))

                            2.5%     97.5%  Odds Ratio
const                   0.250502  0.333109    0.288867
7:00 - 9:59 a.m.        0.742807  0.881227    0.809062
10:00 a.m. - 3:59 p.m.  0.818825  0.932505    0.873818
4:00 - 6:59 p.m.        0.751570  0.874429    0.810675
Monday - Friday         0.854513  0.955735    0.903708
Light rain              0.794159  0.939135    0.863610
Heavy rain              0.708652  1.047268    0.861480
Below 50 km/h           0.567573  0.838812    0.689991
50 km/h                 0.721305  0.847044    0.781650
Severe Congestion       0.749011  0.883345    0.813409
Moderate Congestion     0.752479  0.859402    0.804165
Traffic Signal          0.651324  0.799112    0.721444
Other Control Types     0.649391  0.859965    0.747298
No Control              0.615746  0.831094    0.715362
One-way Carriageway     0.863472  1.005327    0.931704
Two-way Carriageway     1.286172  1.494157    1.386269
Under 15                0.519951  0.716939    0.610552
Above 65  

In [48]:
print('Accuracy:', accuracy_score(
    y_driver, np.rint(m_driver.predict(p_driver, X_driver))))

Accuracy: 0.8628078096910407


In [49]:
pd.DataFrame(confusion_matrix(y_driver, np.rint(m_driver.predict(p_driver, X_driver))), 
             index=['True', 'False'], columns=['True', 'False'])

,True,False
True,41275,0
False,6563,0


In [50]:
X_passen = sm.add_constant(X_passen)
m_passen = sm.Logit(y_passen, X_passen)
r_passen = m_passen.fit()

Optimization terminated successfully.
         Current function value: 0.264141
         Iterations 8


In [51]:
r_passen.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 INJURY   No. Observations:                31905
Model:                          Logit   Df Residuals:                    31878
Method:                           MLE   Df Model:                           26
Date:                Sat, 14 Apr 2018   Pseudo R-squ.:                 0.03258
Time:                        12:17:26   Log-Likelihood:                -8427.4
converged:                       True   LL-Null:                       -8711.2
                                        LLR p-value:                3.295e-103
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -2.2191      0.099    -22.458      0.000      -2.413      -2.025
7:00 - 9:59 a.m.          -0.3003      0.069     -4.338      0.000      -0.436      -0.165
10:00 a.m. - 3:59 p.m.    -0.0674      0.052     -1.295      0.195      -0.169       0.035
4:00 - 6:59 p.m.          -0.2360      0.064     -3.716      0.000      -0.360      -0.112
Monday - Friday           -0.0055      0.046     -0.119      0.905      -0.097       0.086
Light rain                 0.0816      0.063      1.301      0.193      -0.041       0.204
Heavy rain                -0.2622      0.160     -1.636      0.102      -0.576       0.052
Below 50 km/h              0.1142      0.219      0.521      0.602      -0.315       0.544
50 km/h                   -0.1675      0.060     -2.806      0.005      -0.285      -0.051
Severe Congestion          0.0248      0.059      0.420      0.674      -0.091       0.141
Moderate Congestion       -0.1568      0.050     -3.153      0.002      -0.254      -0.059
Traffic Signal            -0.1860      0.063     -2.944      0.003      -0.310      -0.062
Other Control Types       -0.0724      0.110     -0.661      0.509      -0.287       0.142
No Control                -0.3320      0.114     -2.925      0.003      -0.555      -0.110
One-way Carriageway       -0.0948      0.054     -1.759      0.078      -0.200       0.011
Two-way Carriageway        0.1804      0.056      3.212      0.001       0.070       0.291
Under 15                  -1.2595      0.130     -9.711      0.000      -1.514      -1.005
Above 65                   0.7975      0.057     14.063      0.000       0.686       0.909
Male                       0.0174      0.045      0.389      0.697      -0.070       0.105
YEAR_2012                 -0.1094      0.069     -1.578      0.115      -0.245       0.026
YEAR_2013                 -0.0531      0.067     -0.792      0.429      -0.184       0.078
YEAR_2014                 -0.0057      0.068     -0.085      0.932      -0.138       0.127
YEAR_2015                  0.0880      0.065      1.346      0.178      -0.040       0.216
With belt or helmet       -0.0657      0.056     -1.174      0.240      -0.175       0.044
Front nearside            -0.0060      0.060     -0.101      0.919      -0.123       0.111
Standing                   0.2532      0.062      4.059      0.000       0.131       0.375
Others                    -0.1309      0.168     -0.779      0.436      -0.461       0.199
==========================================================================================
"""

In [52]:
p_passen = r_passen.params
c_passen = r_passen.conf_int()
c_passen['OR'] = p_passen
c_passen.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(c_passen))

                            2.5%     97.5%  Odds Ratio
const                   0.089568  0.131937    0.108708
7:00 - 9:59 a.m.        0.646651  0.848222    0.740610
10:00 a.m. - 3:59 p.m.  0.844205  1.035185    0.934830
4:00 - 6:59 p.m.        0.697350  0.894482    0.789789
Monday - Friday         0.907860  1.089352    0.994474
Light rain              0.959541  1.226815    1.084979
Heavy rain              0.561994  1.053198    0.769345
Below 50 km/h           0.729599  1.722189    1.120941
50 km/h                 0.752353  0.950739    0.845749
Severe Congestion       0.913031  1.151055    1.025158
Moderate Congestion     0.775443  0.942393    0.854852
Traffic Signal          0.733597  0.939742    0.830296
Other Control Types     0.750312  1.153046    0.930131
No Control              0.574352  0.896259    0.717473
One-way Carriageway     0.818422  1.010860    0.909566
Two-way Carriageway     1.072860  1.337161    1.197742
Under 15                0.220101  0.365944    0.283804
Above 65  

In [53]:
print('Accuracy:', accuracy_score(
    y_passen, np.rint(m_passen.predict(p_passen, X_passen))))

Accuracy: 0.9223319228960978


In [54]:
pd.DataFrame(confusion_matrix(y_passen, np.rint(m_passen.predict(p_passen, X_passen))), 
             index=['True', 'False'], columns=['True', 'False'])

,True,False
True,29427,0
False,2478,0


In [55]:
X_pedest = sm.add_constant(X_pedest)
m_pedest = sm.Logit(y_pedest, X_pedest)
r_pedest = m_pedest.fit()

Optimization terminated successfully.
         Current function value: 0.522550
         Iterations 6


In [56]:
r_pedest.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 INJURY   No. Observations:                17450
Model:                          Logit   Df Residuals:                    17425
Method:                           MLE   Df Model:                           24
Date:                Sat, 14 Apr 2018   Pseudo R-squ.:                 0.04520
Time:                        12:17:26   Log-Likelihood:                -9118.5
converged:                       True   LL-Null:                       -9550.1
                                        LLR p-value:                8.670e-167
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                -0.0549      0.187     -0.294      0.769      -0.421       0.311
7:00 - 9:59 a.m.                      0.0179      0.057      0.312      0.755      -0.095       0.131
10:00 a.m. - 3:59 p.m.               -0.2186      0.048     -4.520      0.000      -0.313      -0.124
4:00 - 6:59 p.m.                     -0.2585      0.056     -4.654      0.000      -0.367      -0.150
Monday - Friday                       0.0253      0.042      0.602      0.547      -0.057       0.107
Light rain                            0.0428      0.061      0.703      0.482      -0.077       0.162
Heavy rain                            0.2272      0.131      1.734      0.083      -0.030       0.484
Below 50 km/h                        -1.0418      0.204     -5.100      0.000      -1.442      -0.641
50 km/h                              -0.9947      0.175     -5.680      0.000      -1.338      -0.651
Severe Congestion                    -0.0219      0.052     -0.420      0.675      -0.124       0.081
Moderate Congestion                  -0.1327      0.043     -3.100      0.002      -0.217      -0.049
Traffic Signal                       -0.0908      0.055     -1.643      0.100      -0.199       0.018
Other Control Types                  -0.2196      0.086     -2.562      0.010      -0.388      -0.052
No Control                           -0.2400      0.066     -3.649      0.000      -0.369      -0.111
One-way Carriageway                  -0.3520      0.056     -6.292      0.000      -0.462      -0.242
Two-way Carriageway                   0.0322      0.058      0.559      0.576      -0.081       0.145
Under 15                             -0.0615      0.064     -0.963      0.336      -0.187       0.064
Above 65                              0.9650      0.043     22.313      0.000       0.880       1.050
Male                                 -0.1335      0.037     -3.613      0.000      -0.206      -0.061
YEAR_2012                            -0.1104      0.059     -1.871      0.061      -0.226       0.005
YEAR_2013                            -0.0027      0.058     -0.047      0.963      -0.117       0.111
YEAR_2014                            -0.0345      0.059     -0.585      0.558      -0.150       0.081
YEAR_2015                             0.0678      0.058      1.170      0.242      -0.046       0.181
On controlled crossing                0.4065      0.052      7.818      0.000       0.305       0.508
Within 15M of controlled crossing     0.5393      0.073      7.371      0.000       0.396       0.683
=====================================================================================================
"""

In [57]:
p_pedest = r_pedest.params
c_pedest = r_pedest.conf_int()
c_pedest['OR'] = p_pedest
c_pedest.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(c_pedest))

                                       2.5%     97.5%  Odds Ratio
const                              0.656235  1.365375    0.946576
7:00 - 9:59 a.m.                   0.909680  1.139450    1.018104
10:00 a.m. - 3:59 p.m.             0.730925  0.883532    0.803614
4:00 - 6:59 p.m.                   0.692578  0.861025    0.772222
Monday - Friday                    0.944638  1.113481    1.025591
Light rain                         0.926213  1.176214    1.043755
Heavy rain                         0.970861  1.622595    1.255115
Below 50 km/h                      0.236428  0.526531    0.352826
50 km/h                            0.262384  0.521295    0.369837
Severe Congestion                  0.883031  1.083832    0.978293
Moderate Congestion                0.805207  0.952369    0.875702
Traffic Signal                     0.819495  1.017665    0.913220
Other Control Types                0.678629  0.949719    0.802812
No Control                         0.691524  0.894870    0.786654
One-way Ca

In [58]:
print('Accuracy:', accuracy_score(
    y_pedest, np.rint(m_pedest.predict(p_pedest, X_pedest))))

Accuracy: 0.7653868194842407


In [59]:
pd.DataFrame(confusion_matrix(y_pedest, np.rint(m_pedest.predict(p_pedest, X_pedest))), 
             index=['True', 'False'], columns=['True', 'False'])

,True,False
True,13177,142
False,3952,179
